In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import json
from sklearn.preprocessing import MultiLabelBinarizer
df = pd.read_csv("movies_metadata.csv")
genresArray = []
for genre in df['genres']:
    jsonString = str(genre).replace("'", '"')
    genreObj = json.loads(jsonString)
    genresArray.append([])
    for g in genreObj:
        genresArray[-1].append(g['name'])
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(genresArray)


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
if torch.cuda.is_available():
    model.cuda()

genres = ['Mardock Scramble Production Committee', 'Sentai Filmworks', 'Family', 'Drama', 'Thriller', 'Documentary', 'Carousel Productions', 'History', 'Telescene Film Group Productions', 'Western', 'Romance', 'Pulser Productions', 'Vision View Entertainment', 'Foreign', 'Odyssey Media', 'Science Fiction', 'Horror', 'Fantasy', 'GoHands', 'Comedy', 'Action', 'The Cartel', 'BROSTA TV', 'Rogue State', 'Music', 'Crime', 'Adventure', 'TV Movie', 'Animation', 'Mystery', 'War', 'Aniplex']

def title_to_embedding(title):
    inputs = tokenizer(title, return_tensors="pt")
    if torch.cuda.is_available():
        inputs.to('cuda')
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().detach().numpy()

class GenrePredictionModel(nn.Module):
    def __init__(self, input_size, num_genres):
        super(GenrePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 768)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(768, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, num_genres)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        #x = self.softmax(x)
        return x


C:\Users\rizhi\AppData\Local\Temp\ipykernel_27184\2406257748.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_metadata.csv")


In [8]:
cmodel = GenrePredictionModel(768, len(genres))
cmodel.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [26]:

title = "star wars"
embeddings = title_to_embedding(title)
inputs = torch.tensor(embeddings)
outputs = cmodel(inputs)
predicted = outputs > 0
score = 0
for i in range(len(predicted[0])):
    if predicted[0][i]:
        print(mlb.classes_[i])


Action
Adventure
Science Fiction
